In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, MaxPooling1D, Conv1D, SpatialDropout1D, Flatten, GlobalAveragePooling1D, Dropout, AveragePooling1D, Input, Concatenate, Activation, Add, BatchNormalization
from tensorflow.keras.models import Model
!pip install tensorflow-addons
import tensorflow_addons as tfa
import os
!pip install tf2onnx
import tf2onnx
import onnx
import numpy as np
!pip install tensorflow-model-optimization
import tensorflow_model_optimization as tfmot
!pip install onnxruntime
import onnxruntime as ort
!pip install wandb
!wandb login 676572de01bb78bb4574de14deed1133349c0591
import wandb
import os

In [ ]:
from tensorflow import keras
!pip install keras-swa
from swa.tfkeras import SWA
from keras.optimizers import Adam

In [ ]:
# Connect to TPU
print("Tensorflow version " + tf.__version__)

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])  # TPU detection
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
    raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

Tensorflow version 2.12.0
Running on TPU  ['10.76.124.26:8470']


In [ ]:
# loading the test and train dataset as TFRecords
def parse(serialized,signal_shape=(1250,1),label_shape=(1,)):

    features = {'signal': tf.io.FixedLenFeature([], tf.string), 'label': tf.io.FixedLenFeature([], tf.int64)}
    parsed_example = tf.io.parse_single_example(serialized=serialized, features=features)

    signal = parsed_example['signal']
    label = parsed_example['label']

    signal = tf.io.decode_raw(signal, tf.float64)
    signal = tf.reshape(signal, shape=signal_shape)

    label = tf.reshape(label, shape=label_shape)

    return signal, label

AUTOTUNE=tf.data.AUTOTUNE
train_files = tf.io.matching_files('gs://tinyml/train/*train.tfrec')
train_files = tf.random.shuffle(train_files)
shards = tf.data.Dataset.from_tensor_slices(train_files)
train_dataset = shards.interleave(lambda x: tf.data.TFRecordDataset(x), num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.shuffle(buffer_size=10000)
train_dataset = train_dataset.map(parse, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(128)
train_dataset = train_dataset.prefetch(AUTOTUNE)

AUTOTUNE=tf.data.AUTOTUNE
test_files = tf.io.matching_files('gs://tinyml/test/*test.tfrec')
test_files = tf.random.shuffle(test_files)
shards = tf.data.Dataset.from_tensor_slices(test_files)
test_dataset = shards.interleave(lambda x: tf.data.TFRecordDataset(x), num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.map(parse, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(128)
test_dataset = test_dataset.prefetch(AUTOTUNE)

In [ ]:
def create_model(learning_rate):
  model = keras.Sequential([
    keras.layers.Input(shape=(1250, 1)),
    keras.layers.Conv1D(filters=3, kernel_size=85, strides=32, padding='valid', activation=None, use_bias=True),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.Flatten(),

    keras.layers.Dropout(0.3),
    keras.layers.Dense(20),
    keras.layers.ReLU(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(10),
    keras.layers.ReLU(),
    keras.layers.Dense(2),
  ])

  # compiling the model
  AUROC = tf.keras.metrics.AUC(curve='ROC', name = 'AUROC',multi_label = True)
  AUPRC = tf.keras.metrics.AUC(curve='PR', name = 'AUPRC',multi_label = True)
  model.compile(optimizer=Adam(learning_rate=learning_rate),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy', AUROC, AUPRC,tfa.metrics.FBetaScore(average='micro',num_classes=2,beta = 2.0, threshold=0.5)]
            )
  tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True, show_dtype=False,show_layer_names=True, rankdir='TB', expand_nested=False, dpi=96,layer_range=None, show_layer_activations=False)

  return model

In [ ]:
def run_once(count):
  # hyperparameters
  learning_rate = 0.0002
  num_epochs = 100
  batch_size = 32

  # data augmentation settings
  data_aug = True # master flag that enabled the following data augmentations
  mix = False # mixing input and output data
  flip_peak = True # changing the signs of the values in the data (multiplying the data by -1)
  flip_time = False # reversing the signal
  add_noise = True # adding noise to the signal

  # Stochastic Weight Averaging
  start_epoch = 10
  swa = SWA(start_epoch=start_epoch,
        lr_schedule='cyclic',
        swa_lr=0.0001,
        swa_lr2=0.0005,
        swa_freq=5,
        batch_size=batch_size,
        verbose=1)

  with tpu_strategy.scope():
      my_model = create_model(learning_rate)

  save_name = 'random_' + str(count)
  # save_name = 'SWA'
  checkpoint_filepath = './20_10/' + save_name + '/'
  model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
      filepath=checkpoint_filepath,
      save_weights_only=True,
      monitor='val_accuracy',
      mode='max',
      save_best_only=True)

  # training the model
  my_model.fit(train_dataset, validation_data=test_dataset, epochs=num_epochs, batch_size=batch_size, shuffle=True, callbacks=[model_checkpoint_callback])

  # evaluating the trained model
  my_model.load_weights(checkpoint_filepath)
  orig_loss,orig_accuracy,orig_AUROC,orig_AUPRC,orig_fbeta_score = model.evaluate(test_dataset)
  print('Model: ', save_name)
  print(f"loss: {orig_loss} - accuracy: {orig_accuracy} - AUROC: {orig_AUROC} - AUPRC: {orig_AUPRC} - fbeta_score: {orig_fbeta_score}")

  return my_model

In [ ]:
model = run_once(1)